# 2.2 Building chains & LangChain Expression Language (LCEL)
(Modified version of lesson 2 of https://www.deeplearning.ai/short-courses/functions-tools-agents-langchain/)
<br/><br/>

## Setup

### Install dependencies

In [1]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pypdf~=5.1 --upgrade --quiet
%pip install langchain~=0.3.7 langchain_openai~=0.2.6 langchain_community~=0.3.5 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Load environment variables

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

### Setup Models

In [3]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
api_version = "2024-10-01-preview"
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0.0, openai_api_version=api_version)
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-3-large", openai_api_version=api_version)

## Simple Chain

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic} {topic}"
)

output_parser = StrOutputParser()

In [5]:
# Build a chain (creates a RunnableSequence)
chain = prompt | llm | output_parser

In [6]:
chain.invoke({"topic": "bears"})

'Why do bears never get lost?\n\nBecause they always follow the right koala-fications!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [8]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()

/home/vscode/.local/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
retriever.invoke("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [10]:
retriever.invoke("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

### Using a RunnableMap perform parallel actions

In [12]:
from langchain.schema.runnable import RunnableMap

inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

# Invoking the partial chain to see what we get
inputs.invoke({"question": "where did harrison work?"})

/tmp/ipykernel_12850/2193380366.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "context": lambda x: retriever.get_relevant_documents(x["question"]),


{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

In [13]:
# Using the inputs (map) to build a chain
chain = inputs | prompt | llm | output_parser

In [14]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

## Bind - reusing and extending a component (Runnable)

### Bind with tools
Read more here:
* https://python.langchain.com/docs/concepts/tool_calling/#tool-execution
* https://python.langchain.com/docs/how_to/tool_calling/

In [15]:
#Define a tool from a function
from langchain_core.tools import tool

@tool
def weather_search(airport_code: str) -> str:
    """Search for weather given an airport code"""
    return f"Fetching weather for {airport_code}..."

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

function_model = llm.bind_tools([weather_search])
# Above we're using a specialized function model that binds the weather_search tool to the model. We can also use the 
# standard bind function like this (more verbose):
#function_model = model.bind(tools=[convert_to_openai_tool(weather_search)], tool_choice="required")

In [17]:
runnable = prompt | function_model

In [18]:
result = runnable.invoke({"input": "what is the weather in sf"})

In [22]:
print(result.tool_calls)

[{'name': 'weather_search', 'args': {'airport_code': 'SFO'}, 'id': 'call_muz0jFyaL53bDJuW7Xf9WV8D', 'type': 'tool_call'}]


In [24]:
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_muz0jFyaL53bDJuW7Xf9WV8D', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='run-11780983-d692-4496-a128-35d104bfd15b-0' tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'SFO'}, 'id': 'call_muz0jFya

## Fallbacks

In [25]:
from langchain_core.output_parsers import JsonOutputParser


In [26]:
simple_model = llm.bind(
    temperature=0, 
    model="gpt-4o-mini"
)
simple_chain = simple_model | JsonOutputParser()

In [27]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [28]:
# Invoking the model directly and note the response
simple_model.invoke(challenge)

AIMessage(content='```json\n{\n  "poems": [\n    {\n      "title": "Whispers of Dawn",\n      "author": "Emily Rivers",\n      "first_line": "In the quiet hush before the sun"\n    },\n    {\n      "title": "Echoes of the Sea",\n      "author": "Liam Shore",\n      "first_line": "Waves crash upon the ancient rocks"\n    },\n    {\n      "title": "Autumn\'s Embrace",\n      "author": "Sophia Maple",\n      "first_line": "Leaves dance in the crisp, cool air"\n    }\n  ]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 31, 'total_tokens': 158, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual

**Note**: The next line is expected to fail, since the model is too basic.

In [ ]:
simple_chain.invoke(challenge) # EXPECTED TO FAIL

### Try using a more advanced (chat) model instead

In [ ]:
json_model = simple_model.bind(response_format={"type": "json_object"})
json_chain = llm | JsonOutputParser()

In [ ]:
json_chain.invoke(challenge)

### Using a fallback for the simpler chain

In [ ]:
final_chain = simple_chain.with_fallbacks([json_chain])

In [ ]:
final_chain.invoke(challenge)

## Runnable Interface - methods common to all Runnable components

### Chain composition

In [29]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

### Invoke - execute a chain or runnable

In [30]:
chain.invoke({"topic": "bears"})

'Why do bears never get lost?\n\nBecause they always follow the right koala-fications!'

### Batch - run multiple operations in parallel

In [31]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why do bears never get lost?\n\nBecause they always follow the right koala-fications!',
 'Why are frogs so happy? Because they eat whatever bugs them!']

### Streamed response

In [32]:
prompt = ChatPromptTemplate.from_template(
    "Tell me an elaborate joke about {topic}"
)
chain = prompt | llm | output_parser

for chunk in chain.stream({"topic": "bears"}):
    print(chunk, end="|", flush=True)
    

||Sure|,| here's| an| elaborate| bear| joke| for| you|:

|Once| upon| a| time|,| deep| in| the| heart| of| the| forest|,| there| was| a| bear| named| Barry|.| Barry| was| not| your| average| bear|;| he| was| a| bear| with| a| dream|.| He| wanted| to| become| a| stand|-up| comedian|.| Every| day|,| Barry| would| practice| his| jokes| in| front| of| the| other| animals|,| but| they| never| seemed| to| laugh|.| The| squirrels| would| just| chatter| nerv|ously|,| the| deer| would| stare| blank|ly|,| and| the| rabbits| would| hop| away| in| confusion|.

|Determ|ined| to| make| his| dream| come| true|,| Barry| decided| to| venture| into| the| city| to| perform| at| a| comedy| club|.| He| donn|ed| a| sn|az|zy| bow| tie|,| grabbed| his| best| jokes|,| and| set| off| on| his| journey|.| As| he| approached| the| city|,| he| realized| he| needed| a| disguise| to| blend| in| with| the| humans|.| So|,| he| put| on| a| trench| coat| and| a| fed|ora|,| hoping| no| one| would| notice| he| was| a| bear

### There are also async version - read more here:
https://python.langchain.com/docs/how_to/lcel_cheatsheet/